In [1]:
import requests
import json
from tqdm import tqdm
import pandas as pd

In [2]:
url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=JEKSrW3YsUbYoCkbk3oWnYQ4dZmZo9nV71BUEkNkIxIpU6yiMN3xIqMMHyVE9075tTDYa%2Bk3GDOxam5oSmRRmg%3D%3D&pageNo=1&numOfRows=10&type=json&flag=Y&'
req = requests.get(url)
json = req.json()
json

{'DisasterMsg': [{'head': [{'totalCount': 22069},
    {'numOfRows': '10', 'pageNo': '1', 'type': 'JSON'},
    {'RESULT': {'resultCode': 'INFO-0', 'resultMsg': '정상'}}]},
  {'row': [{'create_date': '2020/08/06 10:09:27',
     'location_id': '213',
     'location_name': '전라북도 장수군',
     'md101_sn': '42576',
     'msg': '[행정안전부] 오늘 10시05분 전북(장수) 호우경보, 산사태ㆍ상습침수 등 위험지역 대피, 외출자제 등 안전에 주의바랍니다',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/06 10:09:20',
     'location_id': '139',
     'location_name': '서울특별시 강북구',
     'md101_sn': '42575',
     'msg': '[강북구청]집중호우와 강풍으로 산사태 발생우려가 높으니, 입산금지 바라며 산림 인접 거주자는 안전에 유의하시길 바랍니다.',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/06 10:08:37',
     'location_id': '200',
     'location_name': '전라남도 해남군',
     'md101_sn': '42574',
     'msg': '[해남군청] 현재 우리지역은 강풍 및 풍랑주의보가 발효중이며 많은 비가 내릴 것으로 예상됩니다. 군민 여러분께서는 비닐하우스 등 시설물안전에 유의하시기 바랍니다.',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/06 10:08:22',
     'location_id': '20',
 

In [6]:
def api_crawling():
    url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=JEKSrW3YsUbYoCkbk3oWnYQ4dZmZo9nV71BUEkNkIxIpU6yiMN3xIqMMHyVE9075tTDYa%2Bk3GDOxam5oSmRRmg%3D%3D&pageNo=1&numOfRows=100&type=json&flag=Y&'
    
    req = requests.get(url)
    
    totalCount = req.json()['DisasterMsg'][0]['head'][0]['totalCount']
#    numOfRows = int(req.json()['DisasterMsg'][0]['head'][1]['numOfRows'])
    
    create_date = []
    location_id = []
    location_name = []
    md101_sn = []
    msg = []
    send_platform = []
    
    if (totalCount % 100) == 0:
#        total_page = (totalCount // numOfRows)
        total_page = (totalCount // 100)
    else:
#        total_page = (totalCount // numOfRows) + 1
        total_page = (totalCount // 100) + 1
        
        for pageNo in tqdm(range(1, total_page + 1)):
            url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=JEKSrW3YsUbYoCkbk3oWnYQ4dZmZo9nV71BUEkNkIxIpU6yiMN3xIqMMHyVE9075tTDYa%2Bk3GDOxam5oSmRRmg%3D%3D&numOfRows=100&type=json&flag=Y&pageNo=' + str(pageNo)
            
            req = requests.get(url)
            
            json_data = req.json()
            
            row = json_data['DisasterMsg'][1]['row']
            
            for data in row:
                create_date.append(data['create_date'])
                location_id.append(data['location_id'])
                location_name.append(data['location_name'])
                md101_sn.append(data['md101_sn'])
                msg.append(data['msg'])
                send_platform.append(data['send_platform'])

    msg_api_df = pd.DataFrame({"create_date": create_date, "location_id": location_id, "location_name":location_name,
                               "md101_sn":md101_sn, "msg":msg, "send_platform":send_platform})
    
    return msg_api_df

In [7]:
msg_api_df = api_crawling()

100%|████████████████████████████████████████████████████████████████████████████████| 221/221 [02:17<00:00,  1.61it/s]


In [8]:
msg_api_df

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2020/08/06 10:09:27,213,전라북도 장수군,42576,"[행정안전부] 오늘 10시05분 전북(장수) 호우경보, 산사태ㆍ상습침수 등 위험지역...",cbs
1,2020/08/06 10:09:20,139,서울특별시 강북구,42575,"[강북구청]집중호우와 강풍으로 산사태 발생우려가 높으니, 입산금지 바라며 산림 인접...",cbs
2,2020/08/06 10:08:37,200,전라남도 해남군,42574,[해남군청] 현재 우리지역은 강풍 및 풍랑주의보가 발효중이며 많은 비가 내릴 것으로...,cbs
3,2020/08/06 10:08:22,20,강원도 횡성군,42573,"[횡성군청]호우경보 발령중, 현재 횡성군 관내 많은 비로 일부 하천 범람이 우려되니...",cbs
4,2020/08/06 10:08:04,"2,21,74,113,136,168,202,222,238,6474","강원도 전체,경기도 전체,경상북도 전체,대전광역시 전체,서울특별시 전체,인천광역시 ...",42572,"[중앙재난안전대책본부] 집중호우 위험, 저지대 침수, 비탈면 붕괴 등 위험지역에서 ...",cbs
...,...,...,...,...,...,...
22064,2011/11/20 12:10:41,"3,4,5,6,7,9","강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양군",10,"[국민안전처]20일 12시 동해중부먼바다 풍랑경보, 어선은 출항을 금지하고 출어선은...",cbs
22065,2011/11/18 21:12:20,200,전라남도 해남군,9,"[국민안전처]\n18일21시10분 전남 해남지역 호우경보, 상습침수및위험지역 대피,...",cbs
22066,2011/11/18 11:38:36,217,제주특별자치도 전체,8,"[국민안전처]\n18일11시30분 제주동부지역 호우경보, 상습침수및위험지역 대피,외...",cbs
22067,2011/11/18 08:20:05,217,제주특별자치도 전체,7,"[국민안전처]\n18일8시20분 제주산간,북부지역 호우경보, 상습침수및위험지역 대피...",cbs


In [9]:
# 데이터프레임 'msg_api_df'의 'send_platform'열의 중복 여부 확인
v = msg_api_df.duplicated('send_platform')

In [10]:
for i in range(len(msg_api_df)):
    if v[i] == False:
        print(i)

0
7520
20186


In [14]:
msg_api_df.loc[[0,7520,20186]]

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2020/08/06 10:09:27,213,전라북도 장수군,42576,"[행정안전부] 오늘 10시05분 전북(장수) 호우경보, 산사태ㆍ상습침수 등 위험지역...",cbs
7520,2020/05/11 19:45:42,"2,21,136,168","강원도 전체,경기도 전체,서울특별시 전체,인천광역시 전체",35056,[기상청] 05월11일19:45 북한 강원 평강 북북서쪽 37km 지역 규모4.0 ...,cbs_kma
20186,2017/01/27 09:50:04,3100000000,울산광역시,2336,"〔울산광역재난안전대책본부〕안전안내, 설날 당일 옥동공원묘지 일대 교통혼잡 및 교통사...",SafeOn


In [21]:
msg_api_df[msg_api_df["send_platform"] == 'cbs_kma']

,create_date,location_id,location_name,md101_sn,msg,send_platform
7520,2020/05/11 19:45:42,"2,21,136,168","강원도 전체,경기도 전체,서울특별시 전체,인천광역시 전체",35056,[기상청] 05월11일19:45 북한 강원 평강 북북서쪽 37km 지역 규모4.0 ...,cbs_kma
8146,2020/05/03 22:10:42,"98,179","광주광역시 전체,전라남도 전체",34429,[기상청] 05월03일22:07 전남 해남군 서북서쪽 21km 지역 규모3.1 지진...,cbs_kma
17498,2020/01/30 00:56:43,"74,104,238","경상북도 전체,대구광역시 전체,충청북도 전체",25075,[기상청] 01월30일00:52 경북 상주시 북쪽 20km 지역 규모3.2 지진발생...,cbs_kma
17574,2019/12/30 00:32:26,"53,74,104,119,162","경상남도 전체,경상북도 전체,대구광역시 전체,부산광역시 전체,울산광역시 전체",24998,[기상청] 12월30일00:32 경남 밀양시 동북동쪽 16km 지역 규모3.5 지진...,cbs_kma
17659,2019/10/27 15:40:10,"53,74,104,119,162","경상남도 전체,경상북도 전체,대구광역시 전체,부산광역시 전체,울산광역시 전체",24912,[기상청] 10월27일15:37 경남 창녕군 남쪽 15km 지역 규모3.4 지진발생...,cbs_kma
18053,2019/07/21 11:05:07,"74,104,238","경상북도 전체,대구광역시 전체,충청북도 전체",24517,[기상청] 07월21일11:04 경북 상주시 북북서쪽 11km 지역 규모3.9 지진...,cbs_kma
18160,2019/04/22 05:46:11,"74,104","경상북도 전체,대구광역시 전체",24409,[기상청] 04월22일05:45 경북 울진군 동남동쪽 43km 해역 규모4.0 지진...,cbs_kma
18397,2019/02/10 12:54:35,"74,104","경상북도 전체,대구광역시 전체",24172,[기상청]02월10일12:53 경북 포항시 북구 동북동쪽 58km 해역 규모4.0 ...,cbs_kma


In [22]:
msg_api_df[msg_api_df["send_platform"] == 'SafeOn']

,create_date,location_id,location_name,md101_sn,msg,send_platform
20186,2017/01/27 09:50:04,3100000000,울산광역시,2336,"〔울산광역재난안전대책본부〕안전안내, 설날 당일 옥동공원묘지 일대 교통혼잡 및 교통사...",SafeOn
20197,2017/01/20 14:48:39,4711000000,경상북도 포항시,2325,"[포항시재난안전대책본부]안전안내, 금일 15:00 동해남부앞바다 풍랑경보, 포항시 ...",SafeOn
20233,2016/10/15 20:20:04,3100000000,울산광역시,2289,2016.10.16(일) 예상강수량이 20~60mm로 예상되고 대암댐 자연월류량이 ...,SafeOn
20234,2016/10/15 12:31:51,3100000000,울산광역시,2288,○ 남해안 동부지역 17~18일 대조기로 해수면이 1.5~2.9m 상승되니 저지대 ...,SafeOn
20235,2016/10/15 12:21:23,3100000000,울산광역시,2287,○ 남해안 동부지역 17~18일 대조기로 해수면이 1.5~2.9m 상승되니 저지대 ...,SafeOn
...,...,...,...,...,...,...
22041,2012/01/12 07:10:53,4217000000,강원도 동해시,129,"[풍랑주의보해제] 01.12, 04:00 강원남부앞바다, 강원중부앞바다, 동해중부먼...",SafeOn
22042,2012/01/11 07:29:48,4217000000,강원도 동해시,128,풍랑주의보 발표 01.11. 09:00 발효 동해중부앞바다 사전대비 및 재난예방 철...,SafeOn
22043,2012/01/10 23:27:55,4217000000,강원도 동해시,127,풍랑주의보 발표 동해남부전해상 01월11일 03:00 발효 사전대비 및 재난예방 철...,SafeOn
22044,2012/01/10 22:44:01,4217000000,강원도 동해시,126,풍랑주의보 발표 동해중부먼바다 01월 11일 01:00 발효 사전대비 바랍니다. -...,SafeOn


In [43]:
# 총 379개 행의 'send_platform'열이 'cbs'가 아닌 다른 값을 가짐을 확인
# 분석에 필요 없다고 판단될 경우, 'send_platform'열을 제거

In [ ]:
# msg_api_df = msg_api_df.drop(['send_platform'], axis=column, inplace=True)